In [1]:
import io
from itertools import count
import pymysql
import getpass
import pandas as pd

import networkx as nx
from networkx import Graph as NXGraph
from networkx.drawing.nx_agraph import graphviz_layout

import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import collections

from IPython.display import display, Image

from ipywidgets import interact, FloatSlider, IntSlider, interactive, HBox, Checkbox

In [12]:
conn = pymysql.connect(host = 'localhost',
                       user = 'root', 
                       passwd = 'qwerty123', 
                       database = 'usa_flights') # só muda aqui para flights / usa_flights

cur = conn.cursor()

# Operações

Nó azul são companhias e nós verdes são aeroportos. Uma aresta significa que uma companhia opera num aeroporto.

In [3]:
@interact(DELTA = True, 
          UNITED = True,
          AMERICAN = True,
          WELLS = True,
          SOUTHWEST = True,
          SKY = True,
          WILMIGAN = True,
          BANK = True,
          JETBLUE = True,
          ALASKA = True)
def graph(DELTA, UNITED, AMERICAN, WELLS, SOUTHWEST, SKY, WILMIGAN, BANK, JETBLUE, ALASKA):
    delta = "Companies.name = 'DELTA AIR LINES INC'"
    united = "Companies.name = 'UNITED AIRLINES INC'"
    american = "Companies.name = 'AMERICAN AIRLINES INC'"
    wells = "Companies.name = 'WELLS FARGO BANK NA TRUSTEE'"
    southwest = "Companies.name = 'SOUTHWEST AIRLINES INC'"
    sky = "Companies.name = 'SKY WEST AIRLINES INC'"
    wilmigan = "Companies.name = 'WILMINGTON TRUST CO TRUSTEE'"
    bank = "Companies.name = 'BANK OF UTAH TRUSTEE'"
    jetblue = "Companies.name = 'JETBLUE AIRWAYS CORPORATION'"
    alaska = "Companies.name = 'ALASKA AIRLINES INC'"
    
    include = []
    if DELTA:
        include.append(delta)
    if UNITED:
        include.append(united)
    if AMERICAN:
        include.append(american)
    if WELLS:
        include.append(wells)
    if SOUTHWEST:
        include.append(southwest)
    if SKY:
        include.append(sky)
    if WILMIGAN:
        include.append(wilmigan)
    if BANK:
        include.append(bank)
    if JETBLUE:
        include.append(jetblue)
    if ALASKA:
        include.append(alaska)
    
    if len(include) == 0:
        include = [delta, united, american, wells, southwest, sky, wilmigan, bank, jetblue, alaska]
    
    where = ""
    
    for k in include:
        if k == include[-1]:
            where += k
        else:
            where += k
            where += " OR "
    
    query = '''
    SELECT Companies.name AS Companny,
           CONCAT(Routes.Departure_Airport_id,'-', Routes.Arrival_Airport_id) AS Route,
           COUNT(Aircrafts.N_Number) AS Flights
    FROM Flights
    INNER JOIN Aircrafts ON Aircrafts.n_number = Flights.Aircrafts_id
    INNER JOIN Routes ON Flights.Route_id = Routes.id_route
    INNER JOIN Companies ON Companies.id_company = Aircrafts.Companys_id
    WHERE {0}
    GROUP BY Companies.name, Routes.Departure_Airport_id, Routes.Arrival_Airport_id
    ORDER BY Flights DESC
    ;
    '''.format(where)
    
    conn = pymysql.connect(host = 'localhost',
                           user = 'root', 
                           passwd = 'qwerty123', 
                           database = 'usa_flights')
    
    df = pd.read_sql_query(query, conn)
    G = nx.DiGraph()
    for ind in df.index:
        if df.loc[ind, 'Flights'] >= 100:
            G.add_node(df.loc[ind, 'Companny'], color = 'blue')
            G.add_node(df.loc[ind, 'Route'], color = 'green')
            G.add_edge(df.loc[ind, 'Companny'], df.loc[ind, 'Route'],
                       color = 'red')


    fig, ax = plt.subplots(figsize=(20,20))
    pos = nx.spring_layout(G, scale=1)
    
    nx.draw_networkx(G,
                     pos = pos,
                     ax = ax,
                     node_color = [nx.get_node_attributes(G,'color')[g] for g in G.nodes()],
                     edge_color = [nx.get_edge_attributes(G,'color')[g] for g in G.edges()],)

    plt.show()
         

interactive(children=(Checkbox(value=True, description='DELTA'), Checkbox(value=True, description='UNITED'), C…

In [4]:
def graph(DELTA, UNITED, AMERICAN, WELLS, SOUTHWEST, SKY, WILMIGAN, BANK, JETBLUE, ALASKA):
    delta = "Companies.name = 'DELTA AIR LINES INC'"
    united = "Companies.name = 'UNITED AIRLINES INC'"
    american = "Companies.name = 'AMERICAN AIRLINES INC'"
    wells = "Companies.name = 'WELLS FARGO BANK NA TRUSTEE'"
    southwest = "Companies.name = 'SOUTHWEST AIRLINES INC'"
    sky = "Companies.name = 'SKY WEST AIRLINES INC'"
    wilmigan = "Companies.name = 'WILMINGTON TRUST CO TRUSTEE'"
    bank = "Companies.name = 'BANK OF UTAH TRUSTEE'"
    jetblue = "Companies.name = 'JETBLUE AIRWAYS CORPORATION'"
    alaska = "Companies.name = 'ALASKA AIRLINES INC'"
    
    include = []
    if DELTA:
        include.append(delta)
    if UNITED:
        include.append(united)
    if AMERICAN:
        include.append(american)
    if WELLS:
        include.append(wells)
    if SOUTHWEST:
        include.append(southwest)
    if SKY:
        include.append(sky)
    if WILMIGAN:
        include.append(wilmigan)
    if BANK:
        include.append(bank)
    if JETBLUE:
        include.append(jetblue)
    if ALASKA:
        include.append(alaska)
    
    if len(include) == 0:
        include = [delta, united, american, wells, southwest, sky, wilmigan, bank, jetblue, alaska]
    
    where = ""
    
    for k in include:
        if k == include[-1]:
            where += k
        else:
            where += k
            where += " OR "
    
    query = '''
    SELECT Companies.name AS Companny,
           CONCAT(Routes.Departure_Airport_id,'-', Routes.Arrival_Airport_id) AS Route,
           COUNT(Aircrafts.N_Number) AS Flights
    FROM Flights
    INNER JOIN Aircrafts ON Aircrafts.n_number = Flights.Aircrafts_id
    INNER JOIN Routes ON Flights.Route_id = Routes.id_route
    INNER JOIN Companies ON Companies.id_company = Aircrafts.Companys_id
    WHERE {0}
    GROUP BY Companies.name, Routes.Departure_Airport_id, Routes.Arrival_Airport_id
    ORDER BY Flights DESC
    ;
    '''.format(where)
    
    conn = pymysql.connect(host = 'localhost',
                           user = 'root', 
                           passwd = 'qwerty123', 
                           database = 'usa_flights')

    df = pd.read_sql_query(query, conn)
    G = nx.DiGraph()
    for ind in df.index:
        if df.loc[ind, 'Flights'] >= 100:
            G.add_node(df.loc[ind, 'Companny'], color = 'blue')
            G.add_node(df.loc[ind, 'Route'], color = 'green')
            G.add_edge(df.loc[ind, 'Companny'], df.loc[ind, 'Route'],
                       color = 'red')


    fig, ax = plt.subplots(figsize=(20,20))
    pos = nx.spring_layout(G, scale=1)
    
    nx.draw_networkx(G,
                     pos = pos,
                     ax = ax,
                     node_color = [nx.get_node_attributes(G,'color')[g] for g in G.nodes()],
                     edge_color = [nx.get_edge_attributes(G,'color')[g] for g in G.edges()],)

    plt.show()

DELTA = True 
UNITED = True
AMERICAN = True
WELLS = True
SOUTHWEST = True
SKY = True
WILMIGAN = True
BANK = True
JETBLUE = True
ALASKA = True

ip = interactive(graph, DELTA = DELTA, UNITED = UNITED, AMERICAN = AMERICAN, WELLS = WELLS,
                 SOUTHWEST = SOUTHWEST, SKY = SKY, WILMIGAN = WILMIGAN, BANK = BANK,
                 JETBLUE = JETBLUE, ALASKA = ALASKA)
display(HBox(ip.children[0:5]))
display(HBox(ip.children[5:-1]))
display(ip.children[-1])

Output()

# Rotas

Nós: Aeroportos

Arestas: existe um voo entre os aeroportos

Peso da aresta: quantidade de voos

In [5]:
def graph(qtd):
    query = '''
            SELECT A.Arrival AS Airport,
                   A.Flights + D.Flights AS Total
            FROM (SELECT Routes.Arrival_Airport_id AS Arrival,
                         COUNT(Aircrafts.N_Number) AS Flights
                  FROM Flights
                  INNER JOIN Aircrafts ON Aircrafts.n_number = Flights.Aircrafts_id
                  INNER JOIN Routes ON Flights.Route_id = Routes.id_route
                  INNER JOIN Companies ON Companies.id_company = Aircrafts.Companys_id
                  GROUP BY Routes.Arrival_Airport_id
                  ORDER BY Flights DESC) AS A
            INNER JOIN (SELECT Routes.Departure_Airport_id AS Departure,
                        COUNT(Aircrafts.N_Number) AS Flights
                        FROM Flights
                        INNER JOIN Aircrafts ON Aircrafts.n_number = Flights.Aircrafts_id
                        INNER JOIN Routes ON Flights.Route_id = Routes.id_route
                        INNER JOIN Companies ON Companies.id_company = Aircrafts.Companys_id
                        GROUP BY Routes.Departure_Airport_id
                        ORDER BY Flights DESC) AS D
            ON A.Arrival = D.Departure
            ;
            '''
    
    conn = pymysql.connect(host = 'localhost',
                           user = 'root', 
                           passwd = 'qwerty123', 
                           database = 'usa_flights')

    df = pd.read_sql_query(query, conn)
    G = nx.DiGraph()
    airports = []
    i = 0
    while i < qtd:
        airports.append(df.loc[i, 'Airport'])
        i += 1
    
    query = '''
            SELECT Routes.Arrival_Airport_id AS Arrival,
                   Routes.Departure_Airport_id AS Departure,
                   COUNT(Aircrafts.N_Number) AS Flights
            FROM Flights
            INNER JOIN Aircrafts ON Aircrafts.n_number = Flights.Aircrafts_id
            INNER JOIN Routes ON Flights.Route_id = Routes.id_route
            INNER JOIN Companies ON Companies.id_company = Aircrafts.Companys_id
            GROUP BY Routes.Arrival_Airport_id, Routes.Departure_Airport_id
            ORDER BY Flights DESC
            ;
            '''

    df = pd.read_sql_query(query, conn)
    
    for ind in df.index:
        if df.loc[ind, 'Arrival'] in airports and df.loc[ind, 'Departure'] in airports:
            G.add_node(df.loc[ind, 'Arrival'], color = 'blue')
            G.add_node(df.loc[ind, 'Departure'], color = 'blue')
            G.add_edge(df.loc[ind, 'Arrival'], df.loc[ind, 'Departure'],
                       weight = df.loc[ind, 'Flights'],
                       color = 'red')

    fig, ax = plt.subplots(figsize=(20,20))
    pos = nx.spring_layout(G, scale=1)
    
    nx.draw_networkx(G,
                     pos = pos,
                     ax = ax,
                     node_color = [nx.get_node_attributes(G,'color')[g] for g in G.nodes()],
                     edge_color = [nx.get_edge_attributes(G,'color')[g] for g in G.edges()],)

    plt.show()

qtd = IntSlider(min = 1, max = 30, value = 15, description = "Airports")
interactive(graph, qtd = qtd)

interactive(children=(IntSlider(value=15, description='Airports', max=30, min=1), Output()), _dom_classes=('wi…

# Misturando os dois anteriores...

Companhias com aeroportos em que operam, junto a rota de um aeroporto a outro.

O azul mais escuro são companhias, enquanto o mais claro representa aeroportos. Conexões em azul mais forte representam rotas, enquanto as em azul claro representam que uma companhia opera naquele aeroporto.

## Utilizando os aeroportos com maior fluxo

In [6]:
def graph(comp0, comp1, comp2, comp3, comp4, comp5, comp6, comp7, comp8, comp9, comp10, comp11, airp):
    conn = pymysql.connect(host = 'localhost',
                           user = 'root', 
                           passwd = 'qwerty123', 
                           database = 'usa_flights')
    
    # query para pegar as companhias com maior volume de voos e adicionar numa lista (companies)
    # e passar para a lista include strings da forma "Companies.name = ..." as quais serão, no
    # final, incluidas numa string where que fará as restrições da query final
    query = '''
            SELECT Companies.name AS Companny,
                   COUNT(Aircrafts.N_Number) AS Flights
            FROM Flights
            INNER JOIN Aircrafts ON Aircrafts.n_number = Flights.Aircrafts_id
            INNER JOIN Routes ON Flights.Route_id = Routes.id_route
            INNER JOIN Companies ON Companies.id_company = Aircrafts.Companys_id
            GROUP BY Companies.name
            ORDER BY Flights DESC
            ;
            '''
    df = pd.read_sql_query(query, conn)
    companies = []
    i = 0
    while i < 12:
        companies.append(df.loc[i, 'Companny'])
        i += 1
    
    include = []
    if comp0:
        include.append("Companies.name = '{}'".format(companies[0]))
    if comp1:
        include.append("Companies.name = '{}'".format(companies[1]))
    if comp2:
        include.append("Companies.name = '{}'".format(companies[2]))
    if comp3:
        include.append("Companies.name = '{}'".format(companies[3]))
    if comp4:
        include.append("Companies.name = '{}'".format(companies[4]))
    if comp5:
        include.append("Companies.name = '{}'".format(companies[5]))
    if comp6:
        include.append("Companies.name = '{}'".format(companies[6]))
    if comp7:
        include.append("Companies.name = '{}'".format(companies[7]))
    if comp8:
        include.append("Companies.name = '{}'".format(companies[8]))
    if comp9:
        include.append("Companies.name = '{}'".format(companies[9]))
    if comp10:
        include.append("Companies.name = '{}'".format(companies[10]))
    if comp11:
        include.append("Companies.name = '{}'".format(companies[11]))
    
    if len(include) == 0:
        include = ["Companies.name = '{}'".format(companies[0]),
                   "Companies.name = '{}'".format(companies[1]),
                   "Companies.name = '{}'".format(companies[2]),
                   "Companies.name = '{}'".format(companies[3]),
                   "Companies.name = '{}'".format(companies[4]),
                   "Companies.name = '{}'".format(companies[5]),
                   "Companies.name = '{}'".format(companies[6]),
                   "Companies.name = '{}'".format(companies[7]),
                   "Companies.name = '{}'".format(companies[8]),
                   "Companies.name = '{}'".format(companies[9]),
                   "Companies.name = '{}'".format(companies[10]),
                   "Companies.name = '{}'".format(companies[11])]
    
    where = ""
    for k in include:
        if k == include[-1]:
            where += k
        else:
            where += k
            where += " OR "
    
    # query para ver os aeroportos com maior volume de voo, em seguida será criado uma lista
    # com airp (variável de entrada que diz quantos aeroportos considerar) aeroportos
    query = '''
            SELECT A.Arrival AS Airport,
                   A.Flights + D.Flights AS Total
            FROM (SELECT Routes.Arrival_Airport_id AS Arrival,
                         COUNT(Aircrafts.N_Number) AS Flights
                  FROM Flights
                  INNER JOIN Aircrafts ON Aircrafts.n_number = Flights.Aircrafts_id
                  INNER JOIN Routes ON Flights.Route_id = Routes.id_route
                  INNER JOIN Companies ON Companies.id_company = Aircrafts.Companys_id
                  GROUP BY Routes.Arrival_Airport_id
                  ORDER BY Flights DESC) AS A
            INNER JOIN (SELECT Routes.Departure_Airport_id AS Departure,
                        COUNT(Aircrafts.N_Number) AS Flights
                        FROM Flights
                        INNER JOIN Aircrafts ON Aircrafts.n_number = Flights.Aircrafts_id
                        INNER JOIN Routes ON Flights.Route_id = Routes.id_route
                        INNER JOIN Companies ON Companies.id_company = Aircrafts.Companys_id
                        GROUP BY Routes.Departure_Airport_id
                        ORDER BY Flights DESC) AS D
            ON A.Arrival = D.Departure
            ;
            '''
    
    df = pd.read_sql_query(query, conn)
    airports = []
    i = 0
    while i < airp:
        airports.append(df.loc[i, 'Airport'])
        i += 1
    
    # query final
    query = '''
            SELECT Companies.name AS Companny,
                   Routes.Departure_Airport_id AS Departure,
                   Routes.Arrival_Airport_id AS Arrival,
                   COUNT(Aircrafts.N_Number) AS Flights
            FROM Flights
            INNER JOIN Aircrafts ON Aircrafts.n_number = Flights.Aircrafts_id
            INNER JOIN Routes ON Flights.Route_id = Routes.id_route
            INNER JOIN Companies ON Companies.id_company = Aircrafts.Companys_id
            WHERE {0}
            GROUP BY Companies.name, Routes.Departure_Airport_id, Routes.Arrival_Airport_id
            ORDER BY Flights DESC
            ;
            '''.format(where)
    
    # criando o grafo com apenas as companhias vistas e com os aeroportos mais movimentados
    df = pd.read_sql_query(query, conn)
    G = nx.DiGraph()
    for ind in df.index:
        if df.loc[ind, 'Arrival'] in airports and df.loc[ind, 'Departure'] in airports:
            G.add_node(df.loc[ind, 'Companny'], color = 'lightskyblue')
            G.add_node(df.loc[ind, 'Arrival'], color = 'azure')
            G.add_node(df.loc[ind, 'Departure'], color = 'azure')
            G.add_edge(df.loc[ind, 'Arrival'], df.loc[ind, 'Departure'],
                       weight = df.loc[ind, 'Flights'],
                       color = 'steelblue',
                       relation = 'tem rota para')
            if (df.loc[ind, 'Companny'], df.loc[ind, 'Arrival']) in G.edges():
                G.edges()[(df.loc[ind, 'Companny'], df.loc[ind, 'Arrival'])]['weight'] += df.loc[ind, 'Flights']
            else:
                G.add_edge(df.loc[ind, 'Companny'], df.loc[ind, 'Arrival'],
                           weight = df.loc[ind, 'Flights'],
                           color = 'turquoise',
                           relation = 'opera em')
            if (df.loc[ind, 'Companny'], df.loc[ind, 'Departure']) in G.edges():
                G.edges()[(df.loc[ind, 'Companny'], df.loc[ind, 'Departure'])]['weight'] += df.loc[ind, 'Flights']
            else:
                G.add_edge(df.loc[ind, 'Companny'], df.loc[ind, 'Departure'],
                           weight = df.loc[ind, 'Flights'],
                           color = 'turquoise',
                           relation = 'opera em')
    
    fig, ax = plt.subplots(figsize=(20,20))
    pos = nx.spring_layout(G, scale=1)
    nx.draw_networkx_edge_labels(G, 
                                 pos, 
                                 edge_labels = nx.get_edge_attributes(G,'relation'),
                                 label_pos = 0.5, 
                                 font_size = 9, 
                                 font_color = 'red', 
                                 font_family = 'sans-serif', 
                                 font_weight = 'normal', 
                                 alpha = 1.0, 
                                 bbox = None, 
                                 ax = ax, 
                                 rotate = True)
    
    nx.draw_networkx(G,
                     pos = pos,
                     ax = ax,
                     node_color = [nx.get_node_attributes(G,'color')[g] for g in G.nodes()],
                     edge_color = [nx.get_edge_attributes(G,'color')[g] for g in G.edges()],)
    
    plt.show()

# mesma query das companhias, mas para colocar de label no interact
query = '''
        SELECT Companies.name AS Companny,
               COUNT(Aircrafts.N_Number) AS Flights
        FROM Flights
        INNER JOIN Aircrafts ON Aircrafts.n_number = Flights.Aircrafts_id
        INNER JOIN Routes ON Flights.Route_id = Routes.id_route
        INNER JOIN Companies ON Companies.id_company = Aircrafts.Companys_id
        GROUP BY Companies.name
        ORDER BY Flights DESC
        ;
        '''
df = pd.read_sql_query(query, conn)
companies = []
i = 0
while i < 12:
    companies.append(df.loc[i, 'Companny'])
    i += 1    

# variáveis do interact
comp0 = Checkbox(True, description = companies[0])
comp1 = Checkbox(True, description = companies[1])
comp2 = Checkbox(True, description = companies[2])
comp3 = Checkbox(True, description = companies[3])
comp4 = Checkbox(True, description = companies[4])
comp5 = Checkbox(True, description = companies[5])
comp6 = Checkbox(False, description = companies[6])
comp7 = Checkbox(False, description = companies[7])
comp8 = Checkbox(False, description = companies[8])
comp9 = Checkbox(False, description = companies[9])
comp10 = Checkbox(False, description = companies[10])
comp11 = Checkbox(False, description = companies[11])
airp = IntSlider(min = 1, max = 30, value = 11, description = " ")

# execução do grafo
ip = interactive(graph, comp0 = comp0, comp1 = comp1, comp2 = comp2, comp3 = comp3, comp4 = comp4, comp5 = comp5,
                 comp6 = comp6, comp7 = comp7, comp8 = comp8, comp9 = comp9, comp10 = comp10, comp11 = comp11, airp = airp)
print("Escolha, dentre as 12 maiores companhias em volume de voo, as que são de interesse.")
display(HBox(ip.children[0:3]))
display(HBox(ip.children[3:6]))
display(HBox(ip.children[6:9]))
display(HBox(ip.children[9:12]))
print("Dos aeroportos mais movimentados, quantos você quer considerar?")
display(HBox(ip.children[12:-1]))
print("Para a primeira plotagem faça alguma alteração nas seleções acima.")
display(ip.children[-1])

Escolha, dentre as 12 maiores companhias em volume de voo, as que são de interesse.


Dos aeroportos mais movimentados, quantos você quer considerar?


Para a primeira plotagem faça alguma alteração nas seleções acima.


Output()

## Agora com aeroportos aleatórios

In [7]:
def graph(comp0, comp1, comp2, comp3, comp4, comp5, comp6, comp7, comp8, comp9, comp10, comp11, airp):
    conn = pymysql.connect(host = 'localhost',
                           user = 'root', 
                           passwd = 'qwerty123', 
                           database = 'usa_flights')
    
    # query para pegar as companhias com maior volume de voos e adicionar numa lista (companies)
    # e passar para a lista include strings da forma "Companies.name = ..." as quais serão, no
    # final, incluidas numa string where que fará as restrições da query final
    query = '''
            SELECT Companies.name AS Companny,
                   COUNT(Aircrafts.N_Number) AS Flights
            FROM Flights
            INNER JOIN Aircrafts ON Aircrafts.n_number = Flights.Aircrafts_id
            INNER JOIN Routes ON Flights.Route_id = Routes.id_route
            INNER JOIN Companies ON Companies.id_company = Aircrafts.Companys_id
            GROUP BY Companies.name
            ORDER BY Flights DESC
            ;
            '''
    df = pd.read_sql_query(query, conn)
    companies = []
    i = 0
    while i < 12:
        companies.append(df.loc[i, 'Companny'])
        i += 1
    
    include = []
    if comp0:
        include.append("Companies.name = '{}'".format(companies[0]))
    if comp1:
        include.append("Companies.name = '{}'".format(companies[1]))
    if comp2:
        include.append("Companies.name = '{}'".format(companies[2]))
    if comp3:
        include.append("Companies.name = '{}'".format(companies[3]))
    if comp4:
        include.append("Companies.name = '{}'".format(companies[4]))
    if comp5:
        include.append("Companies.name = '{}'".format(companies[5]))
    if comp6:
        include.append("Companies.name = '{}'".format(companies[6]))
    if comp7:
        include.append("Companies.name = '{}'".format(companies[7]))
    if comp8:
        include.append("Companies.name = '{}'".format(companies[8]))
    if comp9:
        include.append("Companies.name = '{}'".format(companies[9]))
    if comp10:
        include.append("Companies.name = '{}'".format(companies[10]))
    if comp11:
        include.append("Companies.name = '{}'".format(companies[11]))
    
    if len(include) == 0:
        include = ["Companies.name = '{}'".format(companies[0]),
                   "Companies.name = '{}'".format(companies[1]),
                   "Companies.name = '{}'".format(companies[2]),
                   "Companies.name = '{}'".format(companies[3]),
                   "Companies.name = '{}'".format(companies[4]),
                   "Companies.name = '{}'".format(companies[5]),
                   "Companies.name = '{}'".format(companies[6]),
                   "Companies.name = '{}'".format(companies[7]),
                   "Companies.name = '{}'".format(companies[8]),
                   "Companies.name = '{}'".format(companies[9]),
                   "Companies.name = '{}'".format(companies[10]),
                   "Companies.name = '{}'".format(companies[11])]
    
    where = ""
    for k in include:
        if k == include[-1]:
            where += k
        else:
            where += k
            where += " OR "
    
    # query para ver os aeroportos com maior volume de voo, em seguida será criado uma lista
    # com airp (variável de entrada que diz quantos aeroportos considerar) aeroportos
    query = '''
            SELECT A.Arrival AS Airport,
                   A.Flights + D.Flights AS Total
            FROM (SELECT Routes.Arrival_Airport_id AS Arrival,
                         COUNT(Aircrafts.N_Number) AS Flights
                  FROM Flights
                  INNER JOIN Aircrafts ON Aircrafts.n_number = Flights.Aircrafts_id
                  INNER JOIN Routes ON Flights.Route_id = Routes.id_route
                  INNER JOIN Companies ON Companies.id_company = Aircrafts.Companys_id
                  WHERE {0}
                  GROUP BY Routes.Arrival_Airport_id
                  ORDER BY RAND()) AS A
            INNER JOIN (SELECT Routes.Departure_Airport_id AS Departure,
                        COUNT(Aircrafts.N_Number) AS Flights
                        FROM Flights
                        INNER JOIN Aircrafts ON Aircrafts.n_number = Flights.Aircrafts_id
                        INNER JOIN Routes ON Flights.Route_id = Routes.id_route
                        INNER JOIN Companies ON Companies.id_company = Aircrafts.Companys_id
                        WHERE {0}
                        GROUP BY Routes.Departure_Airport_id
                        ORDER BY RAND()) AS D
            ON A.Arrival = D.Departure
            ;
            '''.format(where, where)
    
    df = pd.read_sql_query(query, conn)
    airports = []
    i = 0
    while i < airp:
        airports.append(df.loc[i, 'Airport'])
        i += 1
    
    # query final
    query = '''
            SELECT Companies.name AS Companny,
                   Routes.Departure_Airport_id AS Departure,
                   Routes.Arrival_Airport_id AS Arrival,
                   COUNT(Aircrafts.N_Number) AS Flights
            FROM Flights
            INNER JOIN Aircrafts ON Aircrafts.n_number = Flights.Aircrafts_id
            INNER JOIN Routes ON Flights.Route_id = Routes.id_route
            INNER JOIN Companies ON Companies.id_company = Aircrafts.Companys_id
            WHERE {0}
            GROUP BY Companies.name, Routes.Departure_Airport_id, Routes.Arrival_Airport_id
            ORDER BY Flights DESC
            ;
            '''.format(where)
    
    # criando o grafo com apenas as companhias vistas e com aeroportos aleatórios (mudam a cada plot)
    df = pd.read_sql_query(query, conn)
    G = nx.DiGraph()
    for ind in df.index:
        if df.loc[ind, 'Arrival'] in airports and df.loc[ind, 'Departure'] in airports:
            G.add_node(df.loc[ind, 'Companny'], color = 'lightskyblue')
            G.add_node(df.loc[ind, 'Arrival'], color = 'azure')
            G.add_node(df.loc[ind, 'Departure'], color = 'azure')
            G.add_edge(df.loc[ind, 'Arrival'], df.loc[ind, 'Departure'],
                       weight = df.loc[ind, 'Flights'],
                       color = 'steelblue',
                       relation = 'tem rota para')
            if (df.loc[ind, 'Companny'], df.loc[ind, 'Arrival']) in G.edges():
                G.edges()[(df.loc[ind, 'Companny'], df.loc[ind, 'Arrival'])]['weight'] += df.loc[ind, 'Flights']
            else:
                G.add_edge(df.loc[ind, 'Companny'], df.loc[ind, 'Arrival'],
                           weight = df.loc[ind, 'Flights'],
                           color = 'turquoise',
                           relation = 'opera em')
            if (df.loc[ind, 'Companny'], df.loc[ind, 'Departure']) in G.edges():
                G.edges()[(df.loc[ind, 'Companny'], df.loc[ind, 'Departure'])]['weight'] += df.loc[ind, 'Flights']
            else:
                G.add_edge(df.loc[ind, 'Companny'], df.loc[ind, 'Departure'],
                           weight = df.loc[ind, 'Flights'],
                           color = 'turquoise',
                           relation = 'opera em')
    
    fig, ax = plt.subplots(figsize=(20,20))
    pos = nx.spring_layout(G, scale=1)
    nx.draw_networkx_edge_labels(G, 
                                 pos, 
                                 edge_labels = nx.get_edge_attributes(G,'relation'),
                                 label_pos = 0.5, 
                                 font_size = 9, 
                                 font_color = 'red', 
                                 font_family = 'sans-serif', 
                                 font_weight = 'normal', 
                                 alpha = 1.0, 
                                 bbox = None, 
                                 ax = ax, 
                                 rotate = True)
    
    nx.draw_networkx(G,
                     pos = pos,
                     ax = ax,
                     node_color = [nx.get_node_attributes(G,'color')[g] for g in G.nodes()],
                     edge_color = [nx.get_edge_attributes(G,'color')[g] for g in G.edges()],)
    
    plt.show()

# mesma query das companhias, mas para colocar de label no interact
query = '''
        SELECT Companies.name AS Companny,
               COUNT(Aircrafts.N_Number) AS Flights
        FROM Flights
        INNER JOIN Aircrafts ON Aircrafts.n_number = Flights.Aircrafts_id
        INNER JOIN Routes ON Flights.Route_id = Routes.id_route
        INNER JOIN Companies ON Companies.id_company = Aircrafts.Companys_id
        GROUP BY Companies.name
        ORDER BY Flights DESC
        ;
        '''
df = pd.read_sql_query(query, conn)
companies = []
i = 0
while i < 12:
    companies.append(df.loc[i, 'Companny'])
    i += 1    

# variáveis do interact
comp0 = Checkbox(True, description = companies[0])
comp1 = Checkbox(True, description = companies[1])
comp2 = Checkbox(True, description = companies[2])
comp3 = Checkbox(True, description = companies[3])
comp4 = Checkbox(True, description = companies[4])
comp5 = Checkbox(True, description = companies[5])
comp6 = Checkbox(False, description = companies[6])
comp7 = Checkbox(False, description = companies[7])
comp8 = Checkbox(False, description = companies[8])
comp9 = Checkbox(False, description = companies[9])
comp10 = Checkbox(False, description = companies[10])
comp11 = Checkbox(False, description = companies[11])
airp = IntSlider(min = 1, max = 30, value = 11, description = " ")

# execução do grafo
ip = interactive(graph, comp0 = comp0, comp1 = comp1, comp2 = comp2, comp3 = comp3, comp4 = comp4, comp5 = comp5,
                 comp6 = comp6, comp7 = comp7, comp8 = comp8, comp9 = comp9, comp10 = comp10, comp11 = comp11, airp = airp)
print("Escolha, dentre as 12 maiores companhias em volume de voo, as que são de interesse.")
display(HBox(ip.children[0:3]))
display(HBox(ip.children[3:6]))
display(HBox(ip.children[6:9]))
display(HBox(ip.children[9:12]))
print("Quantos aeroportos você quer considerar? (São aleatórios, logo mudam a cada plot)")
display(HBox(ip.children[12:-1]))
print("Para a primeira plotagem faça alguma alteração nas seleções acima.")
display(ip.children[-1])

Escolha, dentre as 12 maiores companhias em volume de voo, as que são de interesse.


Quantos aeroportos você quer considerar? (São aleatórios, logo mudam a cada plot)


Para a primeira plotagem faça alguma alteração nas seleções acima.


Output()

In [8]:
# a ideia de query parece estar funcionando, ver por que não dá certo o grafo acima

query = '''
        SELECT Companies.name AS Companny,
               COUNT(Aircrafts.N_Number) AS Flights
        FROM Flights
        INNER JOIN Aircrafts ON Aircrafts.n_number = Flights.Aircrafts_id
        INNER JOIN Routes ON Flights.Route_id = Routes.id_route
        INNER JOIN Companies ON Companies.id_company = Aircrafts.Companys_id
        GROUP BY Companies.name
        ORDER BY Flights DESC
        ;
        '''
df = pd.read_sql_query(query, conn)
companies = []
i = 0
while i < 12:
    companies.append(df.loc[i, 'Companny'])
    i += 1

include = []

comp0 = True
comp1 = True
comp2 = True
comp3 = True
comp4 = True
comp5 = True
comp6 = False
comp7 = False
comp8 = False
comp9 = False
comp10 = False
comp11 = False

if comp0:
    include.append("Companies.name = '{}'".format(companies[0]))
if comp1:
    include.append("Companies.name = '{}'".format(companies[1]))
if comp2:
    include.append("Companies.name = '{}'".format(companies[2]))
if comp3:
    include.append("Companies.name = '{}'".format(companies[3]))
if comp4:
    include.append("Companies.name = '{}'".format(companies[4]))
if comp5:
    include.append("Companies.name = '{}'".format(companies[5]))
if comp6:
    include.append("Companies.name = '{}'".format(companies[6]))
if comp7:
    include.append("Companies.name = '{}'".format(companies[7]))
if comp8:
    include.append("Companies.name = '{}'".format(companies[8]))
if comp9:
    include.append("Companies.name = '{}'".format(companies[9]))
if comp10:
    include.append("Companies.name = '{}'".format(companies[10]))
if comp11:
    include.append("Companies.name = '{}'".format(companies[11]))

if len(include) == 0:
    include = ["Companies.name = '{}'".format(companies[0]),
               "Companies.name = '{}'".format(companies[1]),
               "Companies.name = '{}'".format(companies[2]),
               "Companies.name = '{}'".format(companies[3]),
               "Companies.name = '{}'".format(companies[4]),
               "Companies.name = '{}'".format(companies[5]),
               "Companies.name = '{}'".format(companies[6]),
               "Companies.name = '{}'".format(companies[7]),
               "Companies.name = '{}'".format(companies[8]),
               "Companies.name = '{}'".format(companies[9]),
               "Companies.name = '{}'".format(companies[10]),
               "Companies.name = '{}'".format(companies[11])]

where = ""
for k in include:
    if k == include[-1]:
        where += k
    else:
        where += k
        where += " OR "

# query para ver os aeroportos com maior volume de voo, em seguida será criado uma lista
# com airp (variável de entrada que diz quantos aeroportos considerar) aeroportos
query = '''
        SELECT A.Arrival AS Airport,
               A.Companny,
               A.Flights + D.Flights AS Total
        FROM (SELECT Routes.Arrival_Airport_id AS Arrival,
                     COUNT(Aircrafts.N_Number) AS Flights,
                     Companies.name AS Companny
              FROM Flights
              INNER JOIN Aircrafts ON Aircrafts.n_number = Flights.Aircrafts_id
              INNER JOIN Routes ON Flights.Route_id = Routes.id_route
              INNER JOIN Companies ON Companies.id_company = Aircrafts.Companys_id
              WHERE {0}
              GROUP BY Routes.Arrival_Airport_id
              ORDER BY RAND()) AS A
        INNER JOIN (SELECT Routes.Departure_Airport_id AS Departure,
                    COUNT(Aircrafts.N_Number) AS Flights
                    FROM Flights
                    INNER JOIN Aircrafts ON Aircrafts.n_number = Flights.Aircrafts_id
                    INNER JOIN Routes ON Flights.Route_id = Routes.id_route
                    INNER JOIN Companies ON Companies.id_company = Aircrafts.Companys_id
                    WHERE {0}
                    GROUP BY Routes.Departure_Airport_id
                    ORDER BY RAND()) AS D
        ON A.Arrival = D.Departure
        ;
        '''.format(where, where)

pd.read_sql_query(query, conn)

,Airport,Companny,Total
0,ABI,AMERICAN AIRLINES INC,132
1,ONT,AMERICAN AIRLINES INC,1583
2,BIS,DELTA AIR LINES INC,220
3,LNK,DELTA AIR LINES INC,119
4,PHX,AMERICAN AIRLINES INC,13162
5,MIA,AMERICAN AIRLINES INC,5603
6,MLB,AMERICAN AIRLINES INC,256
7,GCK,AMERICAN AIRLINES INC,93
8,SIT,WELLS FARGO BANK NA TRUSTEE,6
9,RHI,SKY WEST AIRLINES INC,58


# Escolhendo quantidade de companhias e aeroportos

In [66]:
def graph(comp, airp):
    print("Plotando a(s) {} companhia(s) junto do(s) {} aeroporto(s) mais movimentados".format(comp, airp))
    conn = pymysql.connect(host = 'localhost',
                           user = 'root', 
                           passwd = 'qwerty123', 
                           database = 'usa_flights')
    
    G = nx.DiGraph()
    
    # query para pegar as companhias com maior volume de voos e adicionar numa lista (companies)
    # e passar para a lista include strings da forma "Companies.name = ..." as quais serão, no
    # final, incluidas numa string where que fará as restrições da query final
    query = '''
            SELECT Companies.name AS Companny,
                   COUNT(Aircrafts.N_Number) AS Flights
            FROM Flights
            INNER JOIN Aircrafts ON Aircrafts.n_number = Flights.Aircrafts_id
            INNER JOIN Routes ON Flights.Route_id = Routes.id_route
            INNER JOIN Companies ON Companies.id_company = Aircrafts.Companys_id
            GROUP BY Companies.name
            ORDER BY Flights DESC
            ;
            '''
    df = pd.read_sql_query(query, conn)
    include = []
    i = 0
    while i < comp:
#         G.add_node(df.loc[i, 'Companny'], color = 'lightskyblue')
        include.append("Companies.name = '{}'".format(df.loc[i, 'Companny']))
        i += 1
    
    where = ""
    for k in include:
        if k == include[-1]:
            where += k
        else:
            where += k
            where += " OR "
    
    # query para ver os aeroportos com maior volume de voo, em seguida será criado uma lista
    # com airp (variável de entrada que diz quantos aeroportos considerar) aeroportos
    query = '''
            SELECT A.Arrival AS Airport,
                   A.Flights + D.Flights AS Total
            FROM (SELECT Routes.Arrival_Airport_id AS Arrival,
                         COUNT(Aircrafts.N_Number) AS Flights
                  FROM Flights
                  INNER JOIN Aircrafts ON Aircrafts.n_number = Flights.Aircrafts_id
                  INNER JOIN Routes ON Flights.Route_id = Routes.id_route
                  INNER JOIN Companies ON Companies.id_company = Aircrafts.Companys_id
                  WHERE {0}
                  GROUP BY Routes.Arrival_Airport_id
                  ORDER BY RAND()) AS A
            INNER JOIN (SELECT Routes.Departure_Airport_id AS Departure,
                        COUNT(Aircrafts.N_Number) AS Flights
                        FROM Flights
                        INNER JOIN Aircrafts ON Aircrafts.n_number = Flights.Aircrafts_id
                        INNER JOIN Routes ON Flights.Route_id = Routes.id_route
                        INNER JOIN Companies ON Companies.id_company = Aircrafts.Companys_id
                        WHERE {0}
                        GROUP BY Routes.Departure_Airport_id
                        ORDER BY RAND()) AS D
            ON A.Arrival = D.Departure
            ORDER BY Total DESC
            ;
            '''.format(where, where)
    
    df = pd.read_sql_query(query, conn)
    airports = []
    i = 0
    while i < airp:
        airports.append(df.loc[i, 'Airport'])
        i += 1
    
    # query final
    query = '''
            SELECT Companies.name AS Companny,
                   Routes.Departure_Airport_id AS Departure,
                   Routes.Arrival_Airport_id AS Arrival,
                   COUNT(Aircrafts.N_Number) AS Flights
            FROM Flights
            INNER JOIN Aircrafts ON Aircrafts.n_number = Flights.Aircrafts_id
            INNER JOIN Routes ON Flights.Route_id = Routes.id_route
            INNER JOIN Companies ON Companies.id_company = Aircrafts.Companys_id
            WHERE {0}
            GROUP BY Companies.name, Routes.Departure_Airport_id, Routes.Arrival_Airport_id
            ORDER BY Flights DESC
            ;
            '''.format(where)
    
    # criando o grafo com apenas as companhias vistas e com aeroportos aleatórios (mudam a cada plot)
    df = pd.read_sql_query(query, conn)
    for ind in df.index:
        if df.loc[ind, 'Arrival'] in airports and df.loc[ind, 'Departure'] in airports:
            # qualquer coisa trocar a linha 28 por essa linha abaixo (ai o grafo inclui as companhias utilizadas)
            G.add_node(df.loc[ind, 'Companny'], color = 'lightskyblue')
            G.add_node(df.loc[ind, 'Arrival'], color = 'azure')
            G.add_node(df.loc[ind, 'Departure'], color = 'azure')
            G.add_edge(df.loc[ind, 'Arrival'], df.loc[ind, 'Departure'],
                       weight = df.loc[ind, 'Flights'],
                       color = 'steelblue',
                       relation = 'tem rota para')
            if (df.loc[ind, 'Companny'], df.loc[ind, 'Arrival']) in G.edges():
                G.edges()[(df.loc[ind, 'Companny'], df.loc[ind, 'Arrival'])]['weight'] += df.loc[ind, 'Flights']
            else:
                G.add_edge(df.loc[ind, 'Companny'], df.loc[ind, 'Arrival'],
                           weight = df.loc[ind, 'Flights'],
                           color = 'turquoise',
                           relation = 'opera em')
            if (df.loc[ind, 'Companny'], df.loc[ind, 'Departure']) in G.edges():
                G.edges()[(df.loc[ind, 'Companny'], df.loc[ind, 'Departure'])]['weight'] += df.loc[ind, 'Flights']
            else:
                G.add_edge(df.loc[ind, 'Companny'], df.loc[ind, 'Departure'],
                           weight = df.loc[ind, 'Flights'],
                           color = 'turquoise',
                           relation = 'opera em')
    
    fig, ax = plt.subplots(figsize=(20,20))
    pos = nx.spring_layout(G, scale=1)
    nx.draw_networkx_edge_labels(G, 
                                 pos, 
                                 edge_labels = nx.get_edge_attributes(G,'relation'),
                                 label_pos = 0.5, 
                                 font_size = 9, 
                                 font_color = 'red', 
                                 font_family = 'sans-serif', 
                                 font_weight = 'normal', 
                                 alpha = 1.0, 
                                 bbox = None, 
                                 ax = ax, 
                                 rotate = True)
    
    nx.draw_networkx(G,
                     pos = pos,
                     ax = ax,
                     node_color = [nx.get_node_attributes(G,'color')[g] for g in G.nodes()],
                     edge_color = [nx.get_edge_attributes(G,'color')[g] for g in G.edges()],)
    
    plt.show()

# variáveis do interact
comp = IntSlider(min = 1, max = 10, value = 5, description = "Companhias")
airp = IntSlider(min = 1, max = 20, value = 10, description = "Aeroportos")

# execução do grafo
ip = interactive(graph, comp = comp, airp = airp)
print("Escolha quantas companhias e quantos aeroportos plotar. Essas escolhas são feitas de acordo com o volume de voo")
display(HBox(ip.children[0:-1]))
print("Para a primeira plotagem faça alguma alteração nas seleções acima.")
display(ip.children[-1])

Escolha quantas companhias e quantos aeroportos plotar. Essas escolhas são feitas de acordo com o volume de voo


Para a primeira plotagem faça alguma alteração nas seleções acima.


Output()

In [22]:
query = '''
        SELECT A.Arrival AS Airport,
               A.Flights + D.Flights AS Total
        FROM (SELECT Routes.Arrival_Airport_id AS Arrival,
                     COUNT(Aircrafts.N_Number) AS Flights
              FROM Flights
              INNER JOIN Aircrafts ON Aircrafts.n_number = Flights.Aircrafts_id
              INNER JOIN Routes ON Flights.Route_id = Routes.id_route
              INNER JOIN Companies ON Companies.id_company = Aircrafts.Companys_id
              GROUP BY Routes.Arrival_Airport_id) AS A
        INNER JOIN (SELECT Routes.Departure_Airport_id AS Departure,
                    COUNT(Aircrafts.N_Number) AS Flights
                    FROM Flights
                    INNER JOIN Aircrafts ON Aircrafts.n_number = Flights.Aircrafts_id
                    INNER JOIN Routes ON Flights.Route_id = Routes.id_route
                    INNER JOIN Companies ON Companies.id_company = Aircrafts.Companys_id
                    GROUP BY Routes.Departure_Airport_id) AS D
        ON A.Arrival = D.Departure
        ;
        '''

In [40]:
query = '''
        SELECT C.name AS Company,
               M.Name AS Manufacturer,
               COUNT(A.N_Number) AS Aircrafts
        FROM companies AS C
        INNER JOIN aircrafts AS A ON C.id_company = A.Companys_id
        INNER JOIN aircraft_models AS AM ON A.Aircraft_Models_id = AM.id_model
        INNER JOIN manufacturer AS M ON AM.Manufacturer_id = M.id_manufacturer
        GROUP BY Company, Manufacturer
        ORDER BY Aircrafts DESC
        ;
        '''

pd.read_sql_query(query, conn)

,Company,Manufacturer,Aircrafts
0,SOUTHWEST AIRLINES INC,BOEING,607
1,UNITED AIRLINES INC,BOEING,540
2,DELTA AIR LINES INC,BOEING,426
3,WELLS FARGO BANK NA TRUSTEE,BOEING,311
4,WELLS FARGO BANK NA TRUSTEE,AIRBUS,306
5,SKY WEST AIRLINES INC,BOMBARDIER INC,286
6,AMERICAN AIRLINES INC,BOEING,282
7,AMERICAN AIRLINES INC,AIRBUS,217
8,DELTA AIR LINES INC,AIRBUS,208
9,WILMINGTON TRUST CO TRUSTEE,BOEING,171


In [36]:
query = '''
        SELECT *
        FROM manufacturer
        ;
        '''

pd.read_sql_query(query, conn)

,id_manufacturer,Name
0,1,MOONEY
1,2,BIGHAM JACK L
2,3,EMBRAER-EMPRESA BRASILEIRA DE
3,4,QUEST AIRCRAFT COMPANY LLC
4,5,YABORA INDUSTRIA AERONAUTICA S
5,6,AMERICAN EUROCOPTER LLC
6,7,CENTRAIR
7,8,BYLER
8,9,BARD JOHN A
9,10,SOCATA


In [42]:
query = '''
        SELECT Companies.name AS Companny,
               COUNT(Aircrafts.N_Number) AS Flights
        FROM Flights
        INNER JOIN Aircrafts ON Aircrafts.n_number = Flights.Aircrafts_id
        INNER JOIN Routes ON Flights.Route_id = Routes.id_route
        INNER JOIN Companies ON Companies.id_company = Aircrafts.Companys_id
        GROUP BY Companies.name
        ORDER BY Flights DESC
        LIMIT 12
        ;
        '''
pd.read_sql_query(query, conn)

,Companny,Flights
0,SOUTHWEST AIRLINES INC,54892
1,WELLS FARGO BANK NA TRUSTEE,46412
2,AMERICAN AIRLINES INC,44474
3,UNITED AIRLINES INC,32749
4,DELTA AIR LINES INC,29371
5,SKY WEST AIRLINES INC,29340
6,WILMINGTON TRUST CO TRUSTEE,11608
7,JETBLUE AIRWAYS CORPORATION,10466
8,MESA AIRLINES INC,9621
9,REPUBLIC AIRWAYS INC,9083


In [61]:
query = '''
        SELECT C.Company AS Company,
               M.Name AS Manufacturer,
               COUNT(A.N_Number) AS Aircrafts
        FROM (SELECT Companies.name AS Company,
                     Companies.id_company AS ID,
                     COUNT(Aircrafts.N_Number) AS Flights
              FROM Flights
              INNER JOIN Aircrafts ON Aircrafts.n_number = Flights.Aircrafts_id
              INNER JOIN Routes ON Flights.Route_id = Routes.id_route
              INNER JOIN Companies ON Companies.id_company = Aircrafts.Companys_id
              GROUP BY Companies.name
              ORDER BY Flights DESC
              LIMIT 12) AS C
        INNER JOIN aircrafts AS A ON C.ID = A.Companys_id
        INNER JOIN aircraft_models AS AM ON A.Aircraft_Models_id = AM.id_model
        INNER JOIN manufacturer AS M ON AM.Manufacturer_id = M.id_manufacturer
        GROUP BY Company, Manufacturer
        ORDER BY Aircrafts DESC
        ;
        '''

pd.read_sql_query(query, conn)

,Company,Manufacturer,Aircrafts
0,SOUTHWEST AIRLINES INC,BOEING,607
1,UNITED AIRLINES INC,BOEING,540
2,DELTA AIR LINES INC,BOEING,426
3,WELLS FARGO BANK NA TRUSTEE,BOEING,311
4,WELLS FARGO BANK NA TRUSTEE,AIRBUS,306
5,SKY WEST AIRLINES INC,BOMBARDIER INC,286
6,AMERICAN AIRLINES INC,BOEING,282
7,AMERICAN AIRLINES INC,AIRBUS,217
8,DELTA AIR LINES INC,AIRBUS,208
9,AMERICAN AIRLINES INC,EMBRAER,201


In [58]:
query = '''
        UPDATE manufacturer
        SET Name = 'EMBRAER'
        WHERE Name = 'EMBRAER-EMPRESA BRASILEIRA DE' OR Name = 'EMBRAER S A' OR Name = 'EMBRAER-EMPRESA BRASILEIRA DE'
        ;
        '''

cur.execute(query)



query = '''
        UPDATE manufacturer
        SET Name = 'MCDONNELL DOUGLAS'
        WHERE Name = 'MCDONNELL DOUGLAS CORPORATION' OR Name = 'MCDONNELL DOUGLAS AIRCRAFT CO'
        ;
        '''

cur.execute(query)



query = '''
        SELECT *
        FROM manufacturer
        ;
        '''

pd.read_sql_query(query, conn)

,id_manufacturer,Name
0,1,MOONEY
1,2,BIGHAM JACK L
2,3,EMBRAER
3,4,QUEST AIRCRAFT COMPANY LLC
4,5,YABORA INDUSTRIA AERONAUTICA S
5,6,AMERICAN EUROCOPTER LLC
6,7,CENTRAIR
7,8,BYLER
8,9,BARD JOHN A
9,10,SOCATA


In [65]:
limite = 12

query = '''
        SELECT C.Company AS Company,
               M.Name AS Manufacturer,
               COUNT(A.N_Number) AS Aircrafts
        FROM (SELECT Companies.name AS Company,
                     Companies.id_company AS ID,
                     COUNT(Aircrafts.N_Number) AS Flights
              FROM Flights
              INNER JOIN Aircrafts ON Aircrafts.n_number = Flights.Aircrafts_id
              INNER JOIN Routes ON Flights.Route_id = Routes.id_route
              INNER JOIN Companies ON Companies.id_company = Aircrafts.Companys_id
              GROUP BY Companies.name
              ORDER BY Flights DESC
              LIMIT {}) AS C
        INNER JOIN aircrafts AS A ON C.ID = A.Companys_id
        INNER JOIN aircraft_models AS AM ON A.Aircraft_Models_id = AM.id_model
        INNER JOIN manufacturer AS M ON AM.Manufacturer_id = M.id_manufacturer
        GROUP BY Company, Manufacturer
        ORDER BY Aircrafts DESC
        ;
        '''.format(limite)

pd.read_sql_query(query, conn)


,Company,Manufacturer,Aircrafts
0,SOUTHWEST AIRLINES INC,BOEING,607
1,UNITED AIRLINES INC,BOEING,540
2,DELTA AIR LINES INC,BOEING,426
3,WELLS FARGO BANK NA TRUSTEE,BOEING,311
4,WELLS FARGO BANK NA TRUSTEE,AIRBUS,306
5,SKY WEST AIRLINES INC,BOMBARDIER INC,286
6,AMERICAN AIRLINES INC,BOEING,282
7,AMERICAN AIRLINES INC,AIRBUS,217
8,DELTA AIR LINES INC,AIRBUS,208
9,AMERICAN AIRLINES INC,EMBRAER,201


,id_model,model,Manufacturer_id
0,1,M20J,1
1,2,MARQUART M 5 CHARGER,2
2,3,MONTANA COYOTE,2
3,4,ERJ 170-200 LR,3
4,5,KODIAK 100,4
5,6,ERJ 170-200 LR,5
6,7,MBB-BK 117 C-2,6
7,8,101C,7
8,9,EVANS VP II,8
9,10,VANS RV-8A,9
